In [1]:
# define constants
DATA_DIR = "content/UCF101"
LABEL_DIR = "content/UCF101_labels"
CACHE_DIR = "content/cache"
FRAMES_PER_CLIP = 10
IMG_SIZE = 224    # video frames would be resized to IMG_SIZE * IMG_SIZE

# Download UCF101 dataset and extract frames
References


*   https://www.kaggle.com/pevogam/starter-ucf101-with-pytorch
*   https://blog.csdn.net/HW140701/article/details/115864277



In [2]:
import os
import sys
import copy
import subprocess
import numpy as np
from prettytable import PrettyTable
from multiprocessing import Pool
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

# !pip install av
# import av
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam
from torchvision import transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader

OS = sys.platform

if not OS == 'win32' and not os.path.exists(CACHE_DIR):
  !mkdir -p $CACHE_DIR
if not OS == 'win32' and not os.path.exists(DATA_DIR):
  !mkdir -p $DATA_DIR
  !wget --no-check-certificate --limit-rate 100M -O UCF101.rar https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
  !unrar x ./UCF101.rar $DATA_DIR > /dev/null
if not OS == 'win32' and not os.path.exists(LABEL_DIR):
  !mkdir $LABEL_DIR
  !wget --no-check-certificate -O UCF101_labels.zip https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip
  !unzip -d $LABEL_DIR ./UCF101_labels.zip > /dev/null

# check if data is ready
if (os.path.exists(DATA_DIR+'/UCF-101/ApplyEyeMakeup') and os.path.exists(LABEL_DIR+'/ucfTrainTestlist')):
  print("ready to go")
else:
  print("Failed to download data\nPlease manually download files from\nhttps://www.crcv.ucf.edu/data/UCF101/UCF101.rar\nand\nhttps://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip")


ready to go


In [3]:
# enable GPU
if torch.cuda.is_available():
  print("Using GPU!")
  device = torch.device("cuda")
else:
  print("Using CPU... this is going to be slow...")
  device = torch.device("cpu")

Using GPU!


In [4]:
# video loader
'''
  params:
    1. the path the video (e.g. "ApplyEyeMakepu/v_ApplyEyeMakeup_g01_c01.avi")
  return:
    sample FRAMES_PER_CLIP frames from the video (evenly distributed along the timeline),
    return a tensor (FRAMES_PER_CLIP x height x width x color_channels) that stores these frames
'''
def video_loader(filename):
  filename = DATA_DIR + '/UCF-101/' + filename
  if not os.path.exists(filename):
    raise Exception("Cannot find file " + filename)
  frames = []
  cap = cv2.VideoCapture(filename)
  while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
      break
    else:
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
      frames.append(frame)
  cap.release()
  if len(frames) < 1:
    raise Exception("Invalid video")
  if len(frames) > FRAMES_PER_CLIP:
    ratio = len(frames) / FRAMES_PER_CLIP
    frames_ = []
    for i in range(FRAMES_PER_CLIP):
      idx = int(i * ratio)
      frames_.append(frames[idx])
    frames = frames_
  frames = np.stack(frames, axis=0)
  return torch.tensor(frames)

In [5]:
v_test = video_loader('ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi')
v_test.shape

torch.Size([10, 224, 224, 3])

In [6]:
# class mapping
class_to_id = {}
id_to_class = {}
with open(LABEL_DIR+'/ucfTrainTestlist/classInd.txt', 'r') as f:
  for line in f:
    line = line.split()
    line[0] = int(line[0]) - 1
    class_to_id[line[1]] = line[0]
    id_to_class[line[0]] = line[1]

In [7]:
# define the UCF101 dataset class
class UCF101(Dataset):
  def __init__(self, _class_to_id, _subset, _video_loader, _transform=None):
    if _subset == 'train':
      train_data = []
      with open(LABEL_DIR+'/ucfTrainTestlist/trainlist01.txt', 'r') as f1:
        for i, line in enumerate(f1):
          if i % 5 == 0:
            # save that for dev set
            continue
          line = line.split()
          train_data.append((int(line[1])-1, line[0]))   # (caption, video_filename)
      self.data = train_data
      f1.close()
    elif _subset == 'test':
      test_data = []
      with open(LABEL_DIR+'/ucfTrainTestlist/testlist01.txt', 'r') as f1:
        for line in f1:
          line_ = line.split('/')
          test_data.append((_class_to_id[line_[0]], line.strip()))
      self.data = test_data
      f1.close()
    elif _subset == 'dev':
      dev_data = []
      with open(LABEL_DIR+'/ucfTrainTestlist/trainlist01.txt', 'r') as f1:
        for i, line in enumerate(f1):
          if i % 5 != 0:
            # the sample is already in training set
            continue
          line = line.split()
          dev_data.append((int(line[1])-1, line[0]))
      self.data = dev_data
      f1.close()
    else:
      raise Exception("_subset should have value 'train', 'test', or 'dev'")
    self.video_loader = _video_loader
    self.transform = _transform
    
  def __len__(self):
    return len(self.data)
    
  def __getitem__(self, idx):
    res = self.data[idx]
    enc_video = self.video_loader(res[1])
    if self.transform is not None:
      enc_video = self.transform(enc_video)
    return (res[0], enc_video)

In [8]:
def custom_collate(batch):
  captions, frames = [], []
  for caption, frame in batch:
    captions.append(caption)    # label of current video sample
    frames.append(frame)        # sampled sequence of frames from the video
  return (
    torch.tensor(captions),
    pad_sequence(frames, batch_first=True)
  )

In [9]:
def tfs(enc_video):
  enc_video = torch.permute(enc_video, [0, 3, 1, 2]).float() / 255
  transfrom = torch.nn.Sequential(
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
  )
  return enc_video

# Load CoAtNet image classifier
Model:
*   CoAtNet (Zihang Dai, et al. 2021)

References:
*   <a href="https://arxiv.org/abs/2106.04803">Research Paper</a>
*   <a href="https://github.com/chinhsuanwu/coatnet-pytorch/blob/master/coatnet.py">Code</a>

In [10]:
# download model
if not OS == 'win32' and not os.path.exists('model'):
  !mkdir -p model/coatnet
  !wget -O model/coatnet/. --no-check-certificate https://github.com/chinhsuanwu/coatnet-pytorch/raw/master/coatnet.py
from model.coatnet.coatnet import CoAtNet

# Encoder + Decoder

In [ ]:
'''
  Reference:
  This model is developed based on
  https://github.com/HHTseng/video-classification
'''


class ResCNNEncoder(nn.Module):
    def __init__(self, fc_hidden1=512, fc_hidden2=512, drop_p=0.3, CNN_embed_dim=300):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(ResCNNEncoder, self).__init__()

        self.fc_hidden1, self.fc_hidden2 = fc_hidden1, fc_hidden2
        self.drop_p = drop_p

        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-1]      # delete the last fc layer.
        self.resnet = nn.Sequential(*modules)
        self.fc1 = nn.Linear(resnet.fc.in_features, fc_hidden1)
        self.bn1 = nn.BatchNorm1d(fc_hidden1, momentum=0.01)
        self.fc2 = nn.Linear(fc_hidden1, fc_hidden2)
        self.bn2 = nn.BatchNorm1d(fc_hidden2, momentum=0.01)
        self.fc3 = nn.Linear(fc_hidden2, CNN_embed_dim)
        
    def forward(self, x_3d):
        cnn_embed_seq = []
        for t in range(x_3d.size(1)):
            # ResNet CNN
            with torch.no_grad():
                x = self.resnet(x_3d[:, t, :, :, :])  # ResNet
                x = x.view(x.size(0), -1)             # flatten output of conv

            # FC layers
            x = self.bn1(self.fc1(x))
            x = F.relu(x)
            x = self.bn2(self.fc2(x))
            x = F.relu(x)
            x = F.dropout(x, p=self.drop_p, training=self.training)
            x = self.fc3(x)

            cnn_embed_seq.append(x)

        # swap time and sample dim such that (sample dim, time dim, CNN latent dim)
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1)
        # cnn_embed_seq: shape=(batch, time_step, input_size)

        return cnn_embed_seq

In [11]:
'''
  Reference:
  This model is developed based on
  https://github.com/HHTseng/video-classification
'''


class CoatCNNEncoder(nn.Module):
    def __init__(self, fc_hidden1=512, fc_hidden2=512, drop_p=0.3, CNN_embed_dim=300):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(CoatCNNEncoder, self).__init__()

        self.fc_hidden1, self.fc_hidden2 = fc_hidden1, fc_hidden2
        self.drop_p = drop_p

        coatnet = CoAtNet((IMG_SIZE, IMG_SIZE), 3, [2,2,3,5,2], [64,96,192,384,768], num_classes=len(id_to_class.keys()))
        modules = list(coatnet.children())[:-1]      # delete the last fc layer.
        self.coatnet = nn.Sequential(*modules)
        self.fc1 = nn.Linear(coatnet.fc.in_features, fc_hidden1)
        self.bn1 = nn.BatchNorm1d(fc_hidden1, momentum=0.01)
        self.fc2 = nn.Linear(fc_hidden1, fc_hidden2)
        self.bn2 = nn.BatchNorm1d(fc_hidden2, momentum=0.01)
        self.fc3 = nn.Linear(fc_hidden2, CNN_embed_dim)
        
    def forward(self, x_3d):
        cnn_embed_seq = []
        for t in range(x_3d.size(1)):
            # ResNet CNN
            x = self.coatnet(x_3d[:, t, :, :, :])   # CoAtNet
            x = x.view(x.size(0), -1)               # flatten output of conv

            # FC layers
            x = self.bn1(self.fc1(x))
            x = F.relu(x)
            x = self.bn2(self.fc2(x))
            x = F.relu(x)
            x = F.dropout(x, p=self.drop_p, training=self.training)
            x = self.fc3(x)

            cnn_embed_seq.append(x)

        # swap time and sample dim such that (sample dim, time dim, CNN latent dim)
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1)
        # cnn_embed_seq: shape=(batch, time_step, input_size)

        return cnn_embed_seq

In [12]:
'''
  Reference:
  This model is developed based on
  https://github.com/HHTseng/video-classification
'''


class DecoderRNN(nn.Module):
    def __init__(self, CNN_embed_dim=300, h_RNN_layers=3, h_RNN=256, h_FC_dim=128, drop_p=0.3, num_classes=50):
        super(DecoderRNN, self).__init__()

        self.RNN_input_size = CNN_embed_dim
        self.h_RNN_layers = h_RNN_layers   # RNN hidden layers
        self.h_RNN = h_RNN                 # RNN hidden nodes
        self.h_FC_dim = h_FC_dim
        self.drop_p = drop_p
        self.num_classes = num_classes

        self.LSTM = nn.LSTM(
            input_size=self.RNN_input_size,
            hidden_size=self.h_RNN,        
            num_layers=h_RNN_layers,       
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.fc1 = nn.Linear(self.h_RNN, self.h_FC_dim)
        self.fc2 = nn.Linear(self.h_FC_dim, self.num_classes)

    def forward(self, x_RNN):
        
        self.LSTM.flatten_parameters()
        RNN_out, (h_n, h_c) = self.LSTM(x_RNN, None)  
        """ h_n shape (n_layers, batch, hidden_size), h_c shape (n_layers, batch, hidden_size) """ 
        """ None represents zero initial hidden state. RNN_out has shape=(batch, time_step, output_size) """

        # FC layers
        x = self.fc1(RNN_out[:, -1, :])   # choose RNN_out at the last time step
        x = F.relu(x)
        x = F.dropout(x, p=self.drop_p, training=self.training)
        x = self.fc2(x)

        return x

In [13]:
# EncoderCNN architecture
CNN_fc_hidden1, CNN_fc_hidden2 = 1024, 768
CNN_embed_dim = 512      # latent dim extracted by 2D CNN
img_x, img_y = 256, 342  # resize video 2d frame size
dropout_p = 0.0          # dropout probability

# DecoderRNN architecture
RNN_hidden_layers = 3
RNN_hidden_nodes = 512
RNN_FC_dim = 256

# image_classifier = ResCNNEncoder(fc_hidden1=CNN_fc_hidden1, fc_hidden2=CNN_fc_hidden2, drop_p=dropout_p, CNN_embed_dim=300).to(device)
image_classifier = CoatCNNEncoder(fc_hidden1=CNN_fc_hidden1, fc_hidden2=CNN_fc_hidden2, drop_p=dropout_p, CNN_embed_dim=300).to(device)
video_classifier = DecoderRNN(CNN_embed_dim=300, h_RNN_layers=RNN_hidden_layers, h_RNN=RNN_hidden_nodes, h_FC_dim=RNN_FC_dim, drop_p=dropout_p, num_classes=len(id_to_class.keys())).to(device)

/opt/conda/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272178570/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
image_classifier.load_state_dict(torch.load(CACHE_DIR+'/image_classifier.pt'))
video_classifier.load_state_dict(torch.load(CACHE_DIR+'/video_classifier.pt'))

In [14]:
def train(encoder_model, decoder_model, optimizer, criterion, train_loader, dev_loader, _device):
  encoder_model.train()
  decoder_model.train()

  losses = []
  scores = []

  with tqdm(total=len(train_loader)+len(dev_loader)) as pbar:
    for batch in train_loader:
      captions, clips = batch
      captions = captions.to(_device)
      clips = clips.to(_device)

      N = clips.shape[0]  # current batch size
      L = clips.shape[1]  # current sequence length

      if N == 1:
        continue

      optimizer.zero_grad()

      # encoded = encoder_model(clips.view(-1, clips.shape[2], clips.shape[3], clips.shape[4]))
      # outputs = decoder_model(encoded.view(N, L, -1))
      encoded = encoder_model(clips)
      outputs = decoder_model(encoded)

      loss = criterion(outputs, captions)
      loss.backward()
      optimizer.step()
      pbar.update(1)
    
    encoder_model.eval()
    decoder_model.eval()
    with torch.no_grad():
      for batch in dev_loader:
        captions, clips = batch
        captions = captions.to(_device)
        clips = clips.to(_device)

        N = clips.shape[0]  # current batch size
        L = clips.shape[1]  # current sequence length

        if N == 1:
          continue

        # encoded = encoder_model(clips.view(-1, clips.shape[2], clips.shape[3], clips.shape[4]))
        # outputs = decoder_model(encoded.view(N, L, -1))
        encoded = encoder_model(clips)
        outputs = decoder_model(encoded)

        loss = criterion(outputs, captions)
        losses.append(loss.item())
        preds = torch.max(outputs, 1)[1]
        score = accuracy_score(captions.cpu().data.squeeze().numpy(), preds.cpu().data.squeeze().numpy())
        scores.append(score)
        pbar.update(1)
  
  return sum(losses) / len(losses), sum(scores) / len(scores)

In [15]:
def eval(encoder_model, decoder_model, criterion, test_loader, _device):
  encoder_model.eval()
  decoder_model.eval()

  losses = []
  scores = []

  with torch.no_grad():
    for batch in tqdm(test_loader):
      captions, clips = batch
      captions = captions.to(_device)
      clips = clips.to(_device)

      N = clips.shape[0]  # current batch size
      L = clips.shape[1]  # current sequence length

      if N == 1:
        continue

      # encoded = encoder_model(clips.view(-1, clips.shape[2], clips.shape[3], clips.shape[4]))
      # outputs = decoder_model(encoded.view(N, L, -1))
      encoded = encoder_model(clips)
      outputs = decoder_model(encoded)

      loss = criterion(outputs, captions)
      losses.append(loss.item())
      preds = torch.max(outputs, 1)[1]
      score = accuracy_score(captions.cpu().data.squeeze().numpy(), preds.cpu().data.squeeze().numpy())
      scores.append(score)
  
  return sum(losses) / len(losses), sum(scores) / len(scores)

In [ ]:
batch_size = 4
max_epoch = 50
early_stopping = 1
learning_rate = 1e-4

train_loader = DataLoader(
    UCF101(class_to_id, 'train', video_loader, _transform=tfs),
    collate_fn=custom_collate,
    batch_size=batch_size,
    shuffle=True
)
dev_loader = DataLoader(
    UCF101(class_to_id, 'dev', video_loader, _transform=tfs),
    collate_fn=custom_collate,
    batch_size=batch_size,
    shuffle=True
)
test_loader = DataLoader(
    UCF101(class_to_id, 'test', video_loader, _transform=tfs),
    collate_fn=custom_collate,
    batch_size=batch_size,
    shuffle=False
)

train_params = list(image_classifier.parameters()) + list(video_classifier.parameters())
optimizer = Adam(train_params, lr=learning_rate)
criterion = CrossEntropyLoss()

best_loss = 99.9
best_acc = 0.0

for epoch in range(max_epoch):
  train_loss, train_acc = train(image_classifier, video_classifier, optimizer, criterion, train_loader, dev_loader, device)
  if train_loss < best_loss:
    torch.save(image_classifier.state_dict(), CACHE_DIR+'/image_classifier.pt')
    torch.save(video_classifier.state_dict(), CACHE_DIR+'/video_classifier.pt')
    best_loss = train_loss
  if train_loss > best_loss * (1 + early_stopping):
    print("Training loss not improving, stop training.")
    break
  print("Epoch {}: train loss = {:.4f}, train acc = {:.4f}".format(epoch, train_loss, train_acc))

  0%|          | 0/2385 [00:00<?, ?it/s]

Epoch 0: train loss = 4.4374, train acc = 0.0168


  0%|          | 0/2385 [00:00<?, ?it/s]

Epoch 1: train loss = 4.2773, train acc = 0.0283


  0%|          | 0/2385 [00:00<?, ?it/s]

...

  0%|          | 0/2385 [00:00<?, ?it/s]

Epoch 28: train loss = 3.2915, train acc = 0.1572


  0%|          | 0/2385 [00:00<?, ?it/s]

Epoch 29: train loss = 3.2220, train acc = 0.1536


  0%|          | 0/2385 [00:00<?, ?it/s]